## Detailed data on commercial banks holdings

In [13]:
import os
import pandas as pd
rootdir = '/'.join(os.getcwd().split('/')[:-1]+['/Fed/FRB_H8/'])
pathd = rootdir + 'FRB_H8_D.csv'
pathf = rootdir + 'FRB_H8_F.csv'
path = rootdir + 'FRB_H8.csv'

H8_all = pd.read_csv(path)
labels = list(H8_all)
items = [label.split(",")[0] for label in labels]

In [14]:
H8_all = pd.read_csv(path   , header = 5, index_col = 0)
H8_dom = pd.read_csv(pathd , header = 5, index_col = 0)
H8_for = pd.read_csv(pathf , header = 5, index_col = 0)

def look_up(text, df):
    for i, item in enumerate(items):
        if item == text:
            return list(df)[i-1] 

def by_country(text):
    col1 = H8_all[look_up(text, H8_all)]
    col2 = H8_dom[look_up(text, H8_dom)]
    col3 = H8_for[look_up(text, H8_for)]
    names = ['All Commercial Banks', 'Domestically Chartered Banks', 'Foreign-related Institutions']
    data = [col1, col2, col3]
    df = pd.DataFrame(dict(zip(names, data)), columns = names)
    df.index = pd.to_datetime(df.index)
    return df

In [17]:
labels

['Series Description',
 'Bank credit, all commercial banks, not seasonally adjusted',
 'Securities in bank credit, all commercial banks, not seasonally adjusted',
 'Treasury and agency securities, all commercial banks, not seasonally adjusted',
 'Treasury and agency securities: Mortgage-backed securities (MBS), all commercial banks, not seasonally adjusted',
 'Treasury and agency securities: Non-MBS, all commercial banks, not seasonally adjusted',
 'Other securities, all commercial banks, not seasonally adjusted',
 'Other securities: Mortgage-backed securities, all commercial banks, not seasonally adjusted',
 'Other securities: Non-MBS, all commercial banks, not seasonally adjusted',
 'Loans and leases in bank credit, all commercial banks, not seasonally adjusted',
 'Commercial and industrial loans, all commercial banks, not seasonally adjusted',
 'Real estate loans, all commercial banks, not seasonally adjusted',
 'Real estate loans: Residential real estate loans, all commercial banks

## Arbitrage spread and the foreign capital

In [ ]:
cash = by_country('Cash assets')
borrow = by_country('Borrowings')
rrp = by_country('Total fed funds sold and securities purcased under agreements to resell at commercial banks')
largedp = by_country('Large time deposits')

df_ar = pd.DataFrame()
df_ar['foreign_ratio'] = cash['Foreign-related Institutions']/cash['All Commercial Banks']
df_ar['forcash'] = cash['Foreign-related Institutions']/1e3 # billions
df_ar['borrow'] = borrow['Foreign-related Institutions']/1e3
df_ar['rrp'] = rrp['Foreign-related Institutions']/1e3
df_ar['largedp'] = largedp['Foreign-related Institutions']/1e3
df_ar['3MTB'] = df['TB3'] * 100 # basis points
df_ar['spread_FFR'] = 100 * df['FedSpread'].rolling(window = 5, center = False).mean()
df_ar['SP500'] = df['SP500'].rolling(window = 5, center = False).mean()

df_ar = df_ar.dropna()

varnames = list(df_ar)
normalized = True

exovars = ['forcash','borrow', 'rrp', 'largedp', '3MTB', 'spread_FFR']

x = df_ar[exovars]
y = df_ar['SP500']

#sb.pairplot(x)
#plt.show()

if normalized: 
    x = normalize(x)
    y = normalize(y)
